Import packages

In [ ]:
import math
import torch
import numpy as np
import gpytorch
from matplotlib import pyplot as plt
from botorch.models import ModelListGP, SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel
import torch
from botorch.acquisition.monte_carlo import MCAcquisitionFunction
from botorch.acquisition.objective import IdentityMCObjective
from botorch.fit import fit_gpytorch_model
from botorch.models import ModelListGP, SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from botorch.optim import optimize_acqf
from botorch.utils.sampling import sample_hypersphere
from botorch.utils.transforms import t_batch_mode_transform
from gpytorch.constraints import Interval
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.quasirandom import SobolEngine
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline
%load_ext autoreload
%autoreload 2

Random data generation

In [ ]:
def getRand(r1, r2, num):
    return (r1 - r2) * torch.rand(num, 1) + r2
    
def gen_data(num_data):
    x = getRand(-3, 3, num_data)
    y = getRand(-3, 3, num_data)
    
    data_fn = lambda x, y: torch.sin(0.15 * 3.1415 * (x + y)) + 1
    '''↓ Uncomment to test a more complicated function ↓'''
    # data_fn = lambda x, y: (x**2+y-11)**2+(x+y**2-7)**2

    latent_fn = data_fn(x, y)
    z = latent_fn.long()  
    ''' .long() will convert z to integers 
        (Do not squeeze, SingleTask GP requires 2xD) '''
    return torch.cat((x,y),dim=1), z, data_fn

train_x, train_y, genfn = gen_data(20) # 20 points
plt.scatter(train_x[:,0].numpy(), train_x[:,1].numpy(), c = train_y)

Test data set (Not implemented)

In [ ]:
def use_known_data():
    pass

In [ ]:
''' Change this varibale to change reshape tensor size '''
res = 20

For reference: actual boundry plot

In [ ]:
test_d1 = np.linspace(-3, 3, res)
test_d2 = np.linspace(-3, 3, res)
test_x_mat, test_y_mat = np.meshgrid(test_d1, test_d2)
test_x_mat, test_y_mat = torch.Tensor(test_x_mat), torch.Tensor(test_y_mat)
test_x = torch.cat((test_x_mat.view(-1,1), test_y_mat.view(-1,1)),dim=1)
test_labels = genfn(test_x_mat, test_y_mat).long().squeeze() 
''' ↑ Use .long() to Convert to integers (0 or 1) ↑ '''
test_y = test_labels.view(-1)
plt.contourf(test_x_mat.numpy(), test_y_mat.numpy(), test_labels.numpy())

Prepare model for training

In [ ]:
octf = Standardize(m=train_y.shape[-1])
train_x = torch.tensor(train_x, dtype=torch.float64) 
train_y = torch.tensor(train_y, dtype=torch.float64)
''' ↑ Set dtype to compatible type ↑ '''
likelihood = GaussianLikelihood(noise_constraint=Interval(1e-6, 1e-3))
model = SingleTaskGP(train_x, train_y, likelihood=likelihood, outcome_transform=octf)
mll = ExactMarginalLogLikelihood(model.likelihood, model)

Training step

In [ ]:
fit_gpytorch_model(mll)

In [ ]:
'''
I don't think it's necessary to put model in eval mode 
↓ But I've left it on in case something subtle is happening ↓
'''
model.eval()
likelihood.eval()

with gpytorch.settings.fast_pred_var(), torch.no_grad():
    test_dist = model(test_x.double())
    pred_means = test_dist.loc

# print(test_x.shape)
# print(test_dist)
# print(pred_means.shape)

Logit plot

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 6))
im = ax.contourf(
    test_x_mat.numpy(), test_y_mat.numpy(), pred_means.numpy().reshape((res, res))
)
fig.colorbar(im, ax=ax)
ax.set_title("Logits: Class " + str(0), fontsize = 20)

Approximate probability with a lightweight sampling step using J samples from the posterior

In [ ]:
pred_samples = test_dist.sample(torch.Size((256,1))).exp()
probabilities = (pred_samples / pred_samples.sum(-1, keepdim=True)).mean(0)*100
''' Don't know why I need x100 '''

Probability plot

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (7, 6))
levels = np.linspace(0, 1.05, 50)

im = ax.contourf(
    test_x_mat.numpy(), test_y_mat.numpy(), probabilities.numpy().reshape((res, res)), levels=levels
)
fig.colorbar(im, ax=ax)
ax.set_title("Probabilities: Class " + str(0), fontsize = 20)

In [ ]:
pred_means = torch.unsqueeze(pred_means, 0) 
''' ↑ Unsqueeze pred_means to plot the boundries ↑ '''
fig, ax = plt.subplots(1,2, figsize=(10, 5))

ax[0].contourf(test_x_mat.numpy(), test_y_mat.numpy(), test_labels.numpy())
ax[0].set_title('True Response', fontsize=20)
ax[1].contourf(test_x_mat.numpy(), test_y_mat.numpy(), pred_means.max(0)[0].reshape((res, res)))
ax[1].set_title('Estimated Response', fontsize=20)

'''
In the future we should probably use something other than contour plot
As this classification model only predicts the boundries (if I am understanding this correctly)
But I don't know hot to plot the predicted boundry
'''